In [2]:
#import packages
import pandas as pd
import numpy as np
import os
from pandasql import sqldf

#### Load the datas

In [3]:
employee = pd.read_csv('./data/employee.csv')
employee.sample(5)

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID
18,109,Daniel,Faviet,DFAVIET,16-Aug-02,FI_ACCOUNT,9000,100,1700
6,204,Hermann,Baer,HBAER,7-Jun-02,PR_REP,10000,70,2700
11,102,Lex,De Haan,LDEHAAN,13-Jan-01,AD_VP,17000,90,1700
35,126,Irene,Mikkilineni,IMIKKILI,28-Sep-06,ST_CLERK,2700,50,2400
33,124,Kevin,Mourgos,KMOURGOS,16-Nov-07,ST_MAN,5800,50,2400


In [4]:
employee.shape

(50, 9)

In [5]:
employee.columns

Index(['EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'EMAIL', 'HIRE_DATE',
       'JOB_ID', 'SALARY', 'DEPARTMENT_ID', 'RESIDENCY_ID'],
      dtype='object')

In [6]:
department = pd.read_csv('./data/department.csv')
department.sample(5)

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,2000
1,20,Marketing,201.0,1800
5,60,IT,103.0,1400
7,80,Sales,145.0,2500


In [7]:
department.shape

(12, 4)

In [8]:
department.columns

Index(['DEPARTMENT_ID', 'DEPARTMENT_NAME', 'MANAGER_ID', 'LOCATION_ID'], dtype='object')

In [9]:
location = pd.read_csv('./data/location.csv')
location.head()

,LOCATION_ID,LOCATION_NAME,STATE,COUNTRY
0,1700,Ikeja,Lagos,Nigeria
1,1800,Ibadan,Oyo,Nigeria
2,1900,Kaduna,Kaduna,Nigeria
3,2000,PH,Rivers,Nigeria
4,2400,Warri,Delta,Nigeria


In [10]:
location.shape

(12, 4)

In [11]:
location.columns

Index(['LOCATION_ID', 'LOCATION_NAME', 'STATE', 'COUNTRY'], dtype='object')

Question 1. 
- Number of remote workers in each department i.e. staff working outside their department location excluding shipping staff?


In [12]:
sql1 = ''' 
        SELECT department.DEPARTMENT_NAME, COUNT(*) AS Total_Remote_Worker
        FROM department 
        JOIN employee ON department.DEPARTMENT_ID = employee.DEPARTMENT_ID
        WHERE employee.RESIDENCY_ID!=department.LOCATION_ID  AND department.DEPARTMENT_NAME!='Shipping'
        GROUP BY department.DEPARTMENT_ID
        '''

sqldf(sql1)

,DEPARTMENT_NAME,Total_Remote_Worker
0,Senior_Executive,1
1,Marketing,1
2,Purchasing,4
3,IT,3


this query shows that no employe works remotly in Finance and Accounting departments!

Question 2.
- Create a manager information dimension table

In [13]:
sql2 =  ''' SELECT department.MANAGER_ID,FIRST_NAME, LAST_NAME,EMAIL,JOB_ID, SALARY, 
           RESIDENCY_ID, department.DEPARTMENT_ID, DEPARTMENT_NAME, LOCATION_ID
           FROM employee JOIN department ON employee.EMPLOYEE_ID = department.MANAGER_ID GROUP BY department.MANAGER_ID
        '''
sqldf(sql2)

,MANAGER_ID,FIRST_NAME,LAST_NAME,EMAIL,JOB_ID,SALARY,RESIDENCY_ID,DEPARTMENT_ID,DEPARTMENT_NAME,LOCATION_ID
0,100.0,Steven,King,SKING,AD_PRES,24000,3000,90,Executive,1700
1,103.0,Alexander,Hunold,AHUNOLD,IT_PROG,9000,1700,60,IT,1400
2,108.0,Nancy,Greenberg,NGREENBE,FI_MGR,12008,1700,100,Finance,1700
3,114.0,Den,Raphaely,DRAPHEAL,PU_MAN,11000,1800,30,Purchasing,1700
4,121.0,Adam,Fripp,AFRIPP,ST_MAN,8200,1900,50,Shipping,2000
5,200.0,Jennifer,Whalen,JWHALEN,AD_ASST,4400,1700,10,Administration,1700
6,201.0,Michael,Hartstein,MHARTSTE,MK_MAN,13000,1000,20,Marketing,1800
7,203.0,Susan,Mavris,SMAVRIS,HR_REP,6500,2400,40,Human Resources,2400
8,204.0,Hermann,Baer,HBAER,PR_REP,10000,2700,70,Public Relations,2700
9,205.0,Shelley,Higgins,SHIGGINS,AC_MGR,12008,1700,110,Accounting,1700


In [14]:
sqldf(sql2).to_csv('./data/manager_info.csv')

Question 3.
- Number of workers earning above the average salary in the organization

In [15]:
sql3 = ''' 
        SELECT AVG(SALARY) as AVG_SALARY, COUNT(EMPLOYEE_ID) AS Number_of_workers
        FROM employee WHERE SALARY > (SELECT AVG(SALARY) FROM employee)
    '''

In [16]:
sqldf(sql3)

,AVG_SALARY,Number_of_workers
0,10500.8,20


Question 4.
Create a new column in the employee table categorizing staff as either early hire or late based on the hire date i.e. if the hire date is greater or equal to the average hire date in the organization that should be late hire else early hire as this will assist in shares distribution.

In [17]:
#change the datatype of the date column to datetime
employee["HIRE_DATE"] = pd.to_datetime(employee["HIRE_DATE"])
employee.head()


,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID
0,198,Donald,OConnell,DOCONNEL,2007-06-21,SH_CLERK,2600,50,1500
1,199,Douglas,Grant,DGRANT,2008-01-13,SH_CLERK,2600,50,1500
2,200,Jennifer,Whalen,JWHALEN,2003-09-17,AD_ASST,4400,10,1700
3,201,Michael,Hartstein,MHARTSTE,2004-02-17,MK_MAN,13000,20,1000
4,202,Pat,Fay,PFAY,2005-08-17,MK_REP,6000,20,1800


In [19]:
employee['year'] = employee['HIRE_DATE'].dt.year
employee['month'] = employee['HIRE_DATE'].dt.month
employee['day'] = employee['HIRE_DATE'].dt.day

In [21]:
sql4 = '''
        SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, EMAIL, HIRE_DATE,
        CASE
            WHEN HIRE_DATE >= (SELECT cast(ROUND(AVG(year)) as int)||'-'||cast(ROUND(AVG(month)) as int)||'-'||cast(ROUND(AVG(day)) as int) FROM employee)
                THEN 'late hire'
            ELSE 'early hire'
                END AS HIRING_STATUS,
        JOB_ID, SALARY, DEPARTMENT_ID, RESIDENCY_ID FROM employee
    '''
sqldf(sql4).head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,HIRE_DATE,HIRING_STATUS,JOB_ID,SALARY,DEPARTMENT_ID,RESIDENCY_ID
0,198,Donald,OConnell,DOCONNEL,2007-06-21 00:00:00.000000,late hire,SH_CLERK,2600,50,1500
1,199,Douglas,Grant,DGRANT,2008-01-13 00:00:00.000000,late hire,SH_CLERK,2600,50,1500
2,200,Jennifer,Whalen,JWHALEN,2003-09-17 00:00:00.000000,early hire,AD_ASST,4400,10,1700
3,201,Michael,Hartstein,MHARTSTE,2004-02-17 00:00:00.000000,early hire,MK_MAN,13000,20,1000
4,202,Pat,Fay,PFAY,2005-08-17 00:00:00.000000,early hire,MK_REP,6000,20,1800


In [22]:
#save the updated table
sqldf(sql4).to_csv('./data/employee_Hiring_status.csv')

#### Tips
Let me check and compaire the average of date calculated using sqldf and pandas

In [23]:
#AVG_Date Using sqldf
avg = sqldf(''' SELECT cast(ROUND(AVG(year)) as int)||'-'||cast(ROUND(AVG(month)) as int)||'-'||cast(ROUND(AVG(day)) as int) AS AVG_Date FROM employee

''')
avg

,AVG_Date
0,2005-6-15


In [24]:
#AVG_Date Using pandas
df=employee["HIRE_DATE"] = pd.to_datetime(employee["HIRE_DATE"]).values.astype(np.int64).mean()
df2=pd.to_datetime(df)
df2

Timestamp('2005-06-03 23:31:12')